# Importing all the necessary libraries and Dataset

In [1]:
from keras.models import load_model
from keras.layers import Activation
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, LeakyReLU, BatchNormalization
from numpy.random import randn
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')
data = pd.read_csv(r"D:\Stuff\CyberSec\Datasets\IDS2018\02-14-2018_processed.csv")
print (data.shape)
print (data.tail())
print (data.columns)

(1044751, 66)
         Dst Port  Protocol  Flow Duration  Tot Bwd Pkts  TotLen Fwd Pkts  \
1044746        80         6       10156986             5             1089   
1044747        80         6            117             0                0   
1044748        80         6        5095331             1                0   
1044749        80         6        5235511             1                0   
1044750       443         6        5807256             4              327   

         TotLen Bwd Pkts  Fwd Pkt Len Max  Fwd Pkt Len Min  Fwd Pkt Len Mean  \
1044746             1923              587                0             217.8   
1044747                0                0                0               0.0   
1044748                0                0                0               0.0   
1044749                0                0                0               0.0   
1044750              145              245                0              54.5   

         Fwd Pkt Len Std  ...  Fwd Seg Siz

## Taking feature and Label columns

In [2]:
X = data.drop(["Label"],axis=1)
y = data["Label"] 
n_cols = data.shape[1]

### Checking if the dataset is clean or not by applying some classfier

In [ ]:
X_true_train, X_true_test, y_true_train, y_true_test = train_test_split(X, y, test_size=0.30, random_state=42)
clf_true = RandomForestClassifier(n_estimators=100)
clf_true.fit(X_true_train,y_true_train)
y_true_pred=clf_true.predict(X_true_test)
print("Base Accuracy:",metrics.accuracy_score(y_true_test, y_true_pred))
print("Base classification report:",metrics.classification_report(y_true_test, y_true_pred))

---
# GAN Train Model
## Defining functions for GAN Model

In [11]:
def generate_latent_points(latent_dim, n_samples):
    x_input = randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input


def generate_fake_samples(generator, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim, n_samples)
    X = generator.predict(x_input)
    y = np.zeros((n_samples, 1))

    return X, y


def generate_real_samples(n):
    X = data.sample(n)
    y = np.ones((n, 1))
    return X, y


def define_generator(latent_dim, n_outputs=n_cols):
    model = Sequential()
    model.add(Dense(15, activation='relu',
              kernel_initializer='he_uniform', input_dim=latent_dim))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(n_outputs, kernel_initializer='he_uniform'))
    model.add(Activation('softplus'))
    return model


generator1 = define_generator(10, n_cols)
generator1.summary()


def define_discriminator(n_inputs=n_cols):
    model = Sequential()
    model.add(Dense(25, activation='relu',
              kernel_initializer='he_uniform', input_dim=n_inputs))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer='adam', metrics=['accuracy'])
    return model


discriminator1 = define_discriminator(n_cols)
discriminator1.summary()


def define_gan(generator, discriminator):
    discriminator.trainable = False
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

# create a line plot of loss for the gan and save to file
def plot_history(d_hist, g_hist):
    plt.subplot(1, 1, 1)
    plt.plot(d_hist, label='d')
    plt.plot(g_hist, label='gen')
    plt.show()
    plt.close()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_40 (Dense)            (None, 15)                165       
                                                                 
 dense_41 (Dense)            (None, 30)                480       
                                                                 
 dense_42 (Dense)            (None, 66)                2046      
                                                                 
 activation_8 (Activation)   (None, 66)                0         
                                                                 
Total params: 2,691
Trainable params: 2,691
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_43 (Dense)           

## Training the GAN Model and exporting the Model

In [12]:
from IPython import display
# train the generator and discriminator
# def train(g_model, d_model, gan_model, latent_dim, n_epochs=100, n_batch=128, n_eval=200):
#     # determine half the size of one batch, for updating the discriminator
#     half_batch = int(n_batch / 2)
#     d_history = []
#     g_history = []
#     first_epoch = 1
#     last_epoch = n_epochs
#     d_history = []
#     g_history = []
#     for epoch in range(n_epochs):
#         x_real, y_real = generate_real_samples(half_batch)
#         x_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
#         d_loss_real, d_real_acc = d_model.train_on_batch(x_real, y_real)
#         d_loss_fake, d_fake_acc = d_model.train_on_batch(x_fake, y_fake)
#         d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
#         x_gan = generate_latent_points(latent_dim, n_batch)
#         y_gan = np.ones((n_batch, 1))
#         g_loss_fake = gan_model.train_on_batch(x_gan, y_gan)
#         print('>%d, d1=%.3f, d2=%.3f d=%.3f g=%.3f' %
#               (epoch+1, d_loss_real, d_loss_fake, d_loss,  g_loss_fake))

#         plot_history(d_history, g_history, epoch, first_epoch, last_epoch)
#         if epoch == n_epochs:
#             print('Final Epoch: d1=%.3f, d2=%.3f d=%.3f g=%.3f' % (d_loss_real, d_loss_fake, d_loss,  g_loss_fake))
#             g_model.save('trained_generated_model.h5')
def train(g_model, d_model, gan_model, latent_dim, n_epochs=1000, n_batch=128, n_eval=200):
    half_batch = int(n_batch / 2)
    d_history = []
    g_history = []
    for epoch in range(n_epochs):
        x_real, y_real = generate_real_samples(half_batch)
        x_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        d_loss_real, d_real_acc = d_model.train_on_batch(x_real, y_real)
        d_loss_fake, d_fake_acc = d_model.train_on_batch(x_fake, y_fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        x_gan = generate_latent_points(latent_dim, n_batch)
        y_gan = np.ones((n_batch, 1))
        g_loss_fake = gan_model.train_on_batch(x_gan, y_gan)
        print('>%d, d1=%.3f, d2=%.3f d=%.3f g=%.3f' %
              (epoch+1, d_loss_real, d_loss_fake, d_loss,  g_loss_fake))
        d_history.append(d_loss)
        g_history.append(g_loss_fake)
        plot_history(d_history, g_history)
        display.clear_output(wait=True)
        g_model.save('trained_generated_model.h5')

# size of the latent space
latent_dim = 10
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# train model
train(generator, discriminator, gan_model, latent_dim)

2/2 [==============================] - 0s 2ms/step


---
# Generating Synthetic Data
## Importing Model and Fixing latent points

In [10]:
from keras.models import load_model
model =load_model("trained_generated_model.h5")

In [11]:
latent_points = generate_latent_points(10, 750)
X = model.predict(latent_points)

column_names = list(data.columns)
data_fake = pd.DataFrame(data=X,  columns=column_names)
data_fake.head()

24/24 [==============================] - 0s 904us/step


,Dst Port,Protocol,Flow Duration,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.704014,0.853304,0.667902,0.686707,0.737717,0.529332,0.450836,0.847271,0.562008,0.847662,...,0.606890,0.879695,0.744318,0.976080,1.004257,0.409393,0.674222,1.078704,0.791925,0.876992
1,1.345297,0.199450,1.066442,0.344328,0.787230,0.733536,0.557622,1.424197,0.254310,1.169641,...,0.287999,1.699136,0.725569,0.317711,0.692511,0.453497,1.562089,2.296783,0.324442,0.915546
2,0.909124,0.295122,0.760085,0.569708,0.357118,0.772467,0.439538,1.311928,0.468788,0.828635,...,0.261600,1.330898,0.725142,0.370107,0.900761,0.566374,1.518644,1.480121,0.529489,0.531063
3,0.624763,0.265930,0.686940,0.492132,0.269012,0.401125,0.951207,0.941023,0.751926,0.410461,...,0.257969,0.987456,0.692985,0.826269,0.730844,0.283812,1.870756,1.515430,0.267227,1.229806
4,0.688606,0.577241,0.488378,0.610429,0.797064,0.556420,0.989390,0.843702,0.744604,0.881842,...,0.518322,0.748466,0.503091,1.038850,0.538541,0.439978,0.775705,1.116047,0.486065,1.162736


In [ ]:
outcome_mean = data_fake.Label.mean()
data_fake['Label'] = data_fake['Label'] > outcome_mean
data_fake["Label"] = data_fake["Label"].astype(int)

In [ ]:
column_names = list(data.columns)
column_names.pop()

features = column_names
label = ['Label']
X_fake_created = data_fake[features]
y_fake_created = data_fake[label]

### See the Metrics of Synthetic data

In [ ]:
X_fake_train, X_fake_test, y_fake_train, y_fake_test = train_test_split(X_fake_created, y_fake_created, test_size=0.30, random_state=42)
clf_fake = RandomForestClassifier(n_estimators=100)
clf_fake.fit(X_fake_train,y_fake_train)
y_fake_pred=clf_fake.predict(X_fake_test)
print("Accuracy of fake data model:",metrics.accuracy_score(y_fake_test, y_fake_pred))
print("Classification report of fake data model:",metrics.classification_report(y_fake_test, y_fake_pred))

### Evaluating Synthetic Data with Classifiers

In [ ]:
from table_evaluator import load_data, TableEvaluator
table_evaluator = TableEvaluator(data, data_fake)
table_evaluator.evaluate(target_col='Label')

---
# Export the Synthetic Data into a CSV File

In [ ]:
data_fake.to_csv('generated_data.csv', index=False)